# Setup

In [1]:
import sys
import time

sys.path.append('../utils')
from pyutils import *

In [2]:
sample="2333133121414131402"

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    puzzle = f.read()

In [4]:
def read_diskmap(dm: str) -> list[int]:
    processed: list[str] = []
    mode: int = 1 # 1 == file, -1 == free space
    files: int = -1
    for digit in dm:
        if not digit.isnumeric():
            continue
        digit = int(digit)
        if mode == 1:
            files += 1
            processed.extend([files] * digit)
        elif mode == -1:
            processed.extend([-1] * digit)
        mode *= -1
    return processed

In [61]:
def block_report(d: list[int]):
    groups = {'used': {}, 'free': {}}
    freeblock = 0
    for n, fid in enumerate(d):
        if fid == -1:
            freeblock += 1
        else:
            if freeblock > 0:
                groups['free'][n - freeblock] = freeblock
                freeblock = 0
            if fid not in groups['used']:
                groups['used'][fid] = []
            groups['used'][fid].append(fid)
    return groups

In [5]:
def optimize_disk(d: list[int]) -> list[int]:
    d = d.copy()
    for n in range(len(d)):
        n = abs(n - len(d)) - 1
        if d[n] != -1:
            leftmost_free = d.index(-1)
            if leftmost_free > n:
                # All sorted, no more work to do
                break
            d[leftmost_free], d[n] = d[n], d[leftmost_free]
    return d

In [6]:
def calc_disk_checksum(d: list[int], show: bool=False) -> str:
    chsum: int = 0
    for n, digit in enumerate(d):
        if show:
            time.sleep(0.005)
            clear_output(wait=True)
            print(f'{repr(n):^8}{repr(digit):^8}{chsum}')
        if digit == -1:
            break
        chsum += n * digit
    return chsum

# Solve

In [15]:
disk: list[int] = read_diskmap(sample)

In [16]:
ta = time.perf_counter()
optimized = optimize_disk(disk)
tb = time.perf_counter()
print(f'Optimized disk of length {len(optimized)} in {tb - ta:.05f}s')

Optimized disk of length 42 in 0.00006s


In [17]:
calc_disk_checksum(optimized, show=False)

1928

In [60]:
' '.join(map(str, disk))

'0 0 -1 -1 -1 1 1 1 -1 -1 -1 2 -1 -1 -1 3 3 3 -1 4 4 -1 5 5 5 5 -1 6 6 6 6 -1 7 7 7 -1 8 8 8 8 9 9'

In [62]:
block_report(disk)

{'used': {0: [0, 0],
  1: [1, 1, 1],
  2: [2],
  3: [3, 3, 3],
  4: [4, 4],
  5: [5, 5, 5, 5],
  6: [6, 6, 6, 6],
  7: [7, 7, 7],
  8: [8, 8, 8, 8],
  9: [9, 9]},
 'free': {2: 3, 8: 3, 12: 3, 18: 1, 21: 1, 26: 1, 31: 1, 35: 1}}